In [13]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

In [21]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
dataset = [train, test]
print('train data: ',train.shape)
print('test data: ',test.shape)

train data:  (891, 12)
test data:  (418, 11)


In [15]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [22]:
for data in dataset:
    logfare = np.log(data['Fare']+1)
    data['logfare'] = logfare
    age_mean = data['Age'].mean()
    age_std = data['Age'].std()
    nmissing_age = data['Age'].isnull().sum()
    age_random = list(map(int, np.random.normal(age_mean, age_std, nmissing_age)))
    data['Age'][data['Age'].isnull()] = age_random
    data['FamilySize'] = data['SibSp']+data['Parch']+1
    data['IsAlone'] = 0
    data.loc[data['FamilySize']==1,'IsAlone'] = 1
    data['CabinType'] = data['Cabin'].apply(lambda x: x[0] if type(x) == str else np.nan )
    data['CabinType'].fillna('No Cabin', inplace=True)
    data['Cabin'].fillna('No Cabin', inplace=True)
    data.loc[data['CabinType']=='G','CabinType'] = 'GT'
    data.loc[data['CabinType']=='T','CabinType'] = 'GT'
    data['Sex'] = data['Sex'].map({'male':1,'female':0})
    
train['Embarked'].fillna('S', inplace=True)
test['Fare'].fillna(test['Fare'].mean(), inplace=True)
test['logfare'].fillna(test['logfare'].mean(), inplace=True)
train['Age'] = (train['Age']-train['Age'].mean())/train['Age'].std()
test['Age'] = (test['Age']-train['Age'].mean())/train['Age'].std()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
X_train = train[['Pclass','Sex','Age','SibSp','Parch','Embarked','logfare','FamilySize','IsAlone','CabinType']]
X_test = test[['Pclass','Sex','Age','SibSp','Parch','Embarked','logfare','FamilySize','IsAlone','CabinType']]
y_train = train['Survived']

In [24]:
X_train = pd.get_dummies(X_train)
X_test= pd.get_dummies(X_test)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [27]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,logfare,FamilySize,IsAlone,Embarked_C,Embarked_Q,Embarked_S,CabinType_A,CabinType_B,CabinType_C,CabinType_D,CabinType_E,CabinType_F,CabinType_GT,CabinType_No Cabin
0,3,1,-0.506008,1,0,2.110213,2,0,0,0,1,0,0,0,0,0,0,0,1
1,1,0,0.573691,1,0,4.280593,2,0,1,0,0,0,0,1,0,0,0,0,0
2,3,0,-0.236083,0,0,2.188856,1,1,0,0,1,0,0,0,0,0,0,0,1
3,1,0,0.371247,1,0,3.990834,2,0,0,0,1,0,0,1,0,0,0,0,0
4,3,1,0.371247,0,0,2.202765,1,1,0,0,1,0,0,0,0,0,0,0,1


In [26]:
X_test.head()

,Pclass,Sex,Age,SibSp,Parch,logfare,FamilySize,IsAlone,Embarked_C,Embarked_Q,Embarked_S,CabinType_A,CabinType_B,CabinType_C,CabinType_D,CabinType_E,CabinType_F,CabinType_GT,CabinType_No Cabin
0,3,1,34.5,0,0,2.178064,1,1,0,1,0,0,0,0,0,0,0,0,1
1,3,0,47.0,1,0,2.079442,2,0,0,0,1,0,0,0,0,0,0,0,1
2,2,1,62.0,0,0,2.369075,1,1,0,1,0,0,0,0,0,0,0,0,1
3,3,1,27.0,0,0,2.268252,1,1,0,0,1,0,0,0,0,0,0,0,1
4,3,0,22.0,1,1,2.586824,3,0,0,0,1,0,0,0,0,0,0,0,1


In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

max_depths = [9,10,11]
for max_depth in max_depths:
    clf = RandomForestClassifier(n_estimators=1000,max_depth=max_depth,random_state=0)
    clf.fit(X_train, y_train)
    yp = clf.predict(X_train)

    accuracy = accuracy_score(y_train, yp)

    print('Accuracy: ', accuracy)

Accuracy:  0.9315375982042648
Accuracy:  0.9450056116722784
Accuracy:  0.9517396184062851


In [31]:
yp_test = clf.predict(X_test)

In [34]:
Pid = test['PassengerId']
submission_df = pd.DataFrame({'PassengerId':Pid,'Survived':yp_test})
submission_df.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [35]:
submission_df.to_csv('submission.csv', index=False)